In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [1]:
!mkdir -p /root/.kaggle

In [4]:
import shutil
shutil.move('kaggle.json', '/root/.kaggle/')

'/root/.kaggle/kaggle.json'

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d darshanthakare/dog-breeds-image-dataset

Dataset URL: https://www.kaggle.com/datasets/darshanthakare/dog-breeds-image-dataset
License(s): MIT
 95% 714M/753M [00:06<00:00, 91.3MB/s]
100% 753M/753M [00:06<00:00, 118MB/s] 


In [7]:
!unzip dog-breeds-image-dataset

Streaming output truncated to the last 5000 lines.
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_116.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_117.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_118.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_119.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_12.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_120.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_121.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_122.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_123.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_124.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_13.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_14.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_15.jpg  
  inflating: Dog Breeds Image Dataset/pekinese/pekinese_16.jpg  
  inflating: Dog Breeds Image 

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

In [9]:
data_dir = 'Dog Breeds Image Dataset' # Corrected directory name
target_size = (224, 224)
batch_size = 32
validation_split = 0.2
seed = 100


train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=target_size,
    batch_size=batch_size,
)

valid_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=target_size,
    batch_size=batch_size,
)

Found 17498 files belonging to 157 classes.
Using 13999 files for training.
Found 17498 files belonging to 157 classes.
Using 3499 files for validation.


In [10]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(224, 224, 3)), # Added Rescaling layer
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=3,
                           activation='relu'),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2,
                              padding='valid'),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(157, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
import os
import tensorflow as tf

data_path = '/content/Dog Breeds Image Dataset'
img_size=(224,224)

# List of invalid image files to exclude
invalid_images = [
    '/content/Dog Breeds Image Dataset/germanshepherd/germanshepherd_7.jpg',
    '/content/Dog Breeds Image Dataset/mix/mix_72.jpg'
]

def is_valid_image(filepath):
    """Checks if the image file is valid (JPEG, PNG, GIF, or BMP)."""
    try:
        with tf.io.gfile.GFile(filepath, 'rb') as f:
            header = f.read(4)
            return (header.startswith(b'\xff\xd8') or  # JPEG
                    header.startswith(b'\x89PNG') or   # PNG
                    header.startswith(b'GIF8') or    # GIF
                    header.startswith(b'BM'))       # BMP
    except Exception:
        return False

def one_hot_encode(image, label):
    return image, tf.one_hot(label, len(class_names))

# Create datasets and filter out invalid images
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    subset='training',
    validation_split=0.2,
    seed=42,
    image_size=img_size,
    label_mode='int'
)
class_names = train_ds.class_names

# Filter invalid images from the dataset
train_data = train_ds.filter(lambda x, y: tf.py_function(is_valid_image, [x], tf.bool)).map(one_hot_encode)


valid_data= tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    subset='validation',
    validation_split=0.2,
    image_size=img_size,
    seed=42,
    label_mode='int',
).filter(lambda x, y: tf.py_function(is_valid_image, [x], tf.bool)).map(one_hot_encode) # Then map to one-hot encode

Found 17498 files belonging to 157 classes.
Using 13999 files for training.
Found 17498 files belonging to 157 classes.
Using 3499 files for validation.


In [14]:
for image,label in train_ds.take(1):
  print(label.numpy())

[ 52  37  27  60 125 111 142 134  60  26  10   0  55  12 156 119  85  37
 118  80 128 155 153  80  96  20  48  92  37  82 146   2]


In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

In [16]:
class_names= train_ds.class_names
class_names

['affenpinscher',
 'afghan_hound',
 'african',
 'airedale',
 'akita',
 'american_terrier',
 'appenzeller',
 'australian_cattledog',
 'australian_terrier',
 'basenji',
 'basset_hound',
 'beagle',
 'bedlington_terrier',
 'bernese_mountain',
 'bichon_frise',
 'blenheim_spaniel',
 'blood_hound',
 'bluetick',
 'border_collie',
 'border_terrier',
 'borzoi',
 'boston_bulldog',
 'bouvier',
 'boxer',
 'brabancon',
 'briard',
 'brittany_spaniel',
 'bull_mastiff',
 'cairn_terrier',
 'cardigan_corgi',
 'caucasian_ovcharka',
 'cavapoo',
 'chesapeake_retriever',
 'chihuahua',
 'chow',
 'clumber',
 'cockapoo',
 'cocker_spaniel',
 'coonhound',
 'cotondetulear',
 'curly_retriever',
 'dachshund',
 'dalmatian',
 'dandie_terrier',
 'dhole',
 'dingo',
 'doberman',
 'english_bulldog',
 'english_hound',
 'english_mastiff',
 'english_setter',
 'english_sheepdog',
 'english_springer',
 'entlebucher',
 'eskimo',
 'flatcoated_retriever',
 'fox_terrier',
 'french_bulldog',
 'german_pointer',
 'germanlonghair_poin

In [17]:
#one hot encoder categorical
# label encoder sparse categorical

len(class_names)

157

In [11]:
model_1.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [19]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
for image,label in train_data.take(1):
  for i in range(12):
    plt.subplot(3,4,i+1)
    # Convert the image tensor to a NumPy array and then to uint8 for plotting
    plt.imshow(image[i].numpy().astype('uint8'))
    # Use argmax on the label tensor to get the index of the predicted class
    plt.title(class_names[label[i].numpy().argmax()])
plt.tight_layout()
plt.show()

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 

<Figure size 1000x800 with 0 Axes>

In [20]:
import os

# List of invalid image files to remove
invalid_images = [
    '/content/Dog Breeds Image Dataset/germanshepherd/germanshepherd_7.jpg',
    '/content/Dog Breeds Image Dataset/mix/mix_72.jpg'
]

# Remove invalid image files
for invalid_file in invalid_images:
    if os.path.exists(invalid_file):
        try:
            os.remove(invalid_file)
            print(f"Removed invalid file: {invalid_file}")
        except Exception as e:
            print(f"Error removing file {invalid_file}: {e}")
    else:
        print(f"Invalid file not found: {invalid_file}")

# Now, recreate the datasets after removing the invalid files
data_path = '/content/Dog Breeds Image Dataset'
img_size=(224,224)

def one_hot_encode(image, label):
    return image, tf.one_hot(label, len(class_names))

# Create datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    subset='training',
    validation_split=0.2,
    seed=42,
    image_size=img_size,
    label_mode='int'
)
class_names = train_ds.class_names

train_data = train_ds.map(one_hot_encode)


valid_data= tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    subset='validation',
    validation_split=0.2,
    image_size=img_size,
    seed=42,
    label_mode='int',
).map(one_hot_encode) # Then map to one-hot encode

print("\nDatasets recreated after removing invalid files.")

Removed invalid file: /content/Dog Breeds Image Dataset/germanshepherd/germanshepherd_7.jpg
Removed invalid file: /content/Dog Breeds Image Dataset/mix/mix_72.jpg
Found 17496 files belonging to 157 classes.
Using 13997 files for training.
Found 17496 files belonging to 157 classes.
Using 3499 files for validation.

Datasets recreated after removing invalid files.


In [22]:
history = model_1.fit(train_data,
                      epochs=5,
                      steps_per_epoch=len(train_data),
                      validation_data=valid_data,
                      validation_steps=len(valid_data))

Epoch 1/5


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 157), output.shape=(None, 157)

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model_1.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy','f1_score'])

history = model_1.fit(
    train_data,
    epochs=5,
    validation_data=valid_data,
)

Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 43s 85ms/step - accuracy: 0.0782 - f1_score: 0.0486 - loss: 4.3583 - val_accuracy: 0.0986 - val_f1_score: 0.0594 - val_loss: 4.2047
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 32s 73ms/step - accuracy: 0.4876 - f1_score: 0.4362 - loss: 2.2751 - val_accuracy: 0.0969 - val_f1_score: 0.0662 - val_loss: 4.9667
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 34s 79ms/step - accuracy: 0.8911 - f1_score: 0.8127 - loss: 0.5353 - val_accuracy: 0.0729 - val_f1_score: 0.0514 - val_loss: 8.1387
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 29s 67ms/step - accuracy: 0.9768 - f1_score: 0.8815 - loss: 0.1850 - val_accuracy: 0.0720 - val_f1_score: 0.0509 - val_loss: 9.1728
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 34s 78ms/step - accuracy: 0.9910 - f1_score: 0.8988 - loss: 0.0816 - val_accuracy: 0.0734 - val_f1_score: 0.0532 - val_loss: 10.3818


In [ ]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

# Apply data augmentation to the training data
train_data_augmented = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

history_augmented = model_1.fit(
    train_data_augmented,
    epochs=5, # You might need more epochs with augmentation
    validation_data=valid_data,
)

base_model = tf.keras.applications.MobileNetV2(input_shape=(img_size[0], img_size[1], 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False # Freeze the base model

# # Add a new classifier on top
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(class_names), activation='softmax')

model_transfer = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

# # Compile and train the transfer learning model
base_learning_rate = 0.0001
model_transfer.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history_transfer = model_transfer.fit(train_data_augmented,
                                      epochs=10,
                                      validation_data=valid_data)

Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 209s 476ms/step - accuracy: 0.0206 - f1_score: 0.0101 - loss: 5.1396 - val_accuracy: 0.0603 - val_f1_score: 0.0371 - val_loss: 4.4126
Epoch 2/5
262/438 ━━━━━━━━━━━━━━━━━━━━ 1:18 446ms/step - accuracy: 0.0386 - f1_score: 0.0210 - loss: 4.5566

In [ ]:
import os

def find_invalid_images(data_path):
    invalid_files = []
    for subdir, _, files in os.walk(data_path):
        for file in files:
            filepath = os.path.join(subdir, file)
            try:
                with tf.io.gfile.GFile(filepath, 'rb') as f:
                    # Check for common image file headers
                    header = f.read(4)
                    if not (header.startswith(b'\xff\xd8') or  # JPEG
                            header.startswith(b'\x89PNG') or   # PNG
                            header.startswith(b'GIF8') or    # GIF
                            header.startswith(b'BM')):       # BMP
                        invalid_files.append(filepath)
            except Exception as e:
                invalid_files.append(filepath)
                print(f"Error processing file {filepath}: {e}")
    return invalid_files

# Find invalid images in the dataset
invalid_images = find_invalid_images(data_path)

# Print the list of invalid files
if invalid_images:
    print("Found invalid image files:")
    for invalid_file in invalid_images:
        print(invalid_file)
else:
    print("No invalid image files found.")

# You may want to remove or quarantine these files before proceeding with training.
# For example, to remove them:
# for invalid_file in invalid_images:
#     os.remove(invalid_file)

In [ ]:
import os

data_dir = 'Dog Breeds Image Dataset'
allowed_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
unsupported_files = []

for root, _, files in os.walk(data_dir):
    for file in files:
        file_extension = os.path.splitext(file)[1].lower()
        if file_extension not in allowed_extensions:
            unsupported_files.append(os.path.join(root, file))

if unsupported_files:
    print("Found unsupported files:")
    for file in unsupported_files:
        print(file)
else:
    print("All files have supported image formats.")

In [ ]:
print("Number of classes:", len(train_data.class_names))
print("Breeds:", train_data.class_names)

In [ ]:
for images, labels in train_data.take(1):
    print("Batch image shape:", images.shape)  # e.g., (32, 224, 224, 3)
    print("Batch label indices:", labels.numpy())
    print("Decoded breed names:", [train_data.class_names[i] for i in labels.numpy()])

In [ ]:
import matplotlib.pyplot as plt

for images, labels in train_data.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(min(9, len(images))):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_data.class_names[labels[i]])
        plt.axis("off")
    plt.show()

In [ ]:
import os
import tensorflow as tf

data_dir = 'Dog Breeds Image Dataset'
unsupported_files_during_load = []

# Create a simple function to attempt loading an image
def load_image(image_path):
    try:
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3)
        return True
    except Exception as e:
        print(f"Error loading {image_path}: {e}")
        return False

for root, _, files in os.walk(data_dir):
    for file in files:
        file_path = os.path.join(root, file)
        if not load_image(file_path):
            unsupported_files_during_load.append(file_path)

if unsupported_files_during_load:
    print("\nFiles that caused loading errors:")
    for file in unsupported_files_during_load:
        print(file)
else:
    print("\nAll files loaded successfully.")